# Pydantic fundamentals

### Start without pandantic

In [1]:
# defined a class Person (blueprint)
class Person:
    def __init__(self, name, gender, age):
        self.name = name 
        self.gender = gender
        self.age = age 

# instantiated an instance from the Person class
person1 = Person(name = "Kokchun", age = 34, gender = "M")
person1

In [2]:
person1.name, person1.age

('Kokchun', 34)

In [3]:
person2 = Person(name = 3213+3213, gender = True, age ="minus ten")
person2

### Validation of our Person class

In [6]:
class Person:
    def __init__(self, name, gender, age):
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name 
        self.gender = gender
        self.age = age 
try:        
    Person(name = 3.1415, age = 34, gender = "M")
except TypeError as err:
    print(err)


name must be of type str not <class 'float'>


In [3]:
class Person:
    def __init__(self, name: str, gender: str, age: int): 
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name 
        
        self.gender = gender
        self.age = age 
    
    # gettter
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value: int):
        if not isinstance(value, int):
            raise TypeError(f"age must be of type int not {type(age)} that you have provided")
        if value < 0 or value > 125:
            raise ValueError(f"age must be between 0 and 125, not {value} that you provided")
        
        self._age = value
    

person5 = Person(name = "Bella", gender = "F", age = 3)
person5.age

3

In [4]:
try:
    Person(name = "Bella", gender = "F", age = -3)
except ValueError as err:
    print(err)

age must be between 0 and 125, not -3 that you provided


In [5]:

try:
    Person(name = "Bella", gender = "F", age = 3121)
except ValueError as err:
    print(err)

age must be between 0 and 125, not 3121 that you provided


### Validate using pydantic

In [9]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    gender: str
    age: int

person6 = Person(name = "Christina", gender = "F", age = 29)
person6

Person(name='Christina', gender='F', age=29)

In [10]:
person6.age

29

In [11]:
person6.age = 36
person6

Person(name='Christina', gender='F', age=36)

In [12]:
person6.age = "thirty"
person6

Person(name='Christina', gender='F', age='thirty')

In [13]:
from pydantic import ValidationError

try:
    Person(name = 12344, gender = "F", age = 29)
except ValidationError as err:
    print(err)

1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=12344, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [14]:
Person(name = "Nina", gender = "F", age = 29)

Person(name='Nina', gender='F', age=29)

In [ ]:
from pydantic import ConfigDict

class Person(BaseModel):
    name: str
    gender: str
    age: int
    
    model_config = ConfigDict(validate_assignment=True)
    
person7 = Person(name = "Nina", gender = "F", age = 29)

try:
    Person(name = 12344, gender = "F", age = 29)
except ValidationError as err:
    print(err)
    
person7.age = "thirty"
person7


In [19]:
try:
    Person(name = 12344, gender = 2.9584, age = -4)
except ValidationError as err:
    print(err)

2 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=12344, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=2.9584, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [ ]:
try:
    Person(name = "sofia", gender = "F", age = -4)
except ValidationError as err:
    print(err)

### Age validation

In [22]:
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal ["M", "F", "O"]
    age: int = Field(gt = -1, lt = 125)
    
    model_config = ConfigDict(validate_assignment=True)
    
try:
    Person(name = "sofia", gender = "Fem", age = -4)
except ValidationError as err:
    print(err)

2 validation errors for Person
gender
  Input should be 'M', 'F' or 'O' [type=literal_error, input_value='Fem', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


### Serialization and deserialization

In [24]:
person7.model_dump()

{'name': 'Nina', 'gender': 'F', 'age': 29}

In [31]:
import json
with open("person.json", "w") as file:
    json.dump(person7.model_dump(), file)

In [32]:
# deserialization
with open("person.json", "r") as file:
    json_data = file.read()

Person.model_validate_json(json_data)

Person(name='Nina', gender='F', age=29)